In [1]:
!pip3 install langchain langchain-community langchain-ollama langchain-openai ollama chromadb
!pip3 install pandas streamlit python-dotenv pypdf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
import uuid, re
import pandas as pd
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
df_gemini_answers = pd.read_csv('qa-base/qa_gemini.csv')

df_gemini_answers

,ID pergunta,Pergunta,Resposta,Alucinação,Motivo
0,1,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,Factual,Artigo 139 do código penal não possui parágrafos.
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...",Input,No código penal diz que é um a três anos ou mu...
2,3,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,Não,NaN
3,4,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,Factual,Está no inciso I.
4,5,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",Não,NaN
5,6,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,Não,NaN
6,7,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,Não,NaN
7,8,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,Factual,é de dois a cinco anos.
8,9,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,Não,NaN
9,10,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,Factual,"Tanto o artigo 36 no livro do Código Penal, qu..."


In [5]:
df_deepseek_answers = pd.read_csv('qa-base/qa_deepseek.csv')

df_deepseek_answers

,ID pergunta,Pergunta,Resposta,Alucinação,Motivo
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",Factual,"Em difamação, a exceção da verdade somente se ..."
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",Não,NaN
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\r\n\r\nConfigura-se quando alguém...",Não,NaN
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",Factual,"A pena é detenção, de seis meses a três anos, ..."
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,Não,NaN
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Não,NaN
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\r\...,Factual,Art. 110 fala sobre o aumento do prazo de pres...
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\r\n\r\nDefinição: Pri...",Input,"A pergunta pede as penalidades relacionadas, m..."
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\r\n\...",Não,NaN
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,Não,NaN


In [6]:
model_gpt = ChatOpenAI(model="gpt-4.1-mini-2025-04-14") 
model_llama = OllamaLLM(model="llama3")

In [7]:
answerer_llm = ["deepseek", "gemini"]

## Alucinações de input

In [8]:
def has_input_hallucination_llm(question, answer, model):
    inputHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    prompt = ChatPromptTemplate.from_template(inputHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"question": question, "answer": answer})
    return result

In [9]:
def answerer(answer_mod, model, answer_platform):
    for index, row in answer_mod.iterrows():
        question = row['Pergunta']
        answer = row['Resposta']
        input_hallucination_llm = has_input_hallucination_llm(question, answer, model)
        answer_platform.append({"question":question, "answer":answer, "input_hallucination_reason":input_hallucination_llm})
        print(input_hallucination_llm)
    
    return answer_platform

### LLama 3

In [10]:
def txt_processing_llama(answer_platform):
    for i in answer_platform:
        llama_content = i["input_hallucination_reason"]

        if llama_content:
            #Pega o 0 se houver alucinação ou 1, se não houver
            has_input_hallu = re.match(r"^([01])[.\s]", llama_content)

            #Retira o 0 ou o 1 para salvar na coluna input_hallucination_reason
            i["input_hallucination_reason"] = re.sub(r"^[01]\.?\s*", "", llama_content.strip())

            i["has_input_hallucination"] = has_input_hallu[0]

In [11]:
has_hallucination_llama_ds = []
has_hallucination_llama_gm = []

#### DeepSeek

In [ ]:
input_hallu = answerer(df_deepseek_answers, model_llama, has_hallucination_llama_ds)
txt_processing_llama(has_hallucination_llama_ds)

0. A resposta gerada é relevante e direciona-se à pergunta feita sobre o crime de difamação, tratando do tema especificamente e fornecendo informações precisas sobre a legislação brasileira. Não há saída do tópico da pergunta, portanto não há alucinação de input.
0
Explicação: A resposta gerada é relevante e não foge do tópico da pergunta feita pelo usuário, tratando especificamente sobre a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência.
0. A resposta gerada está relacionada ao tema da pergunta e não apresenta qualquer tipo de alucinação, mantendo o foco no assunto de falsificação de documento público.
0. A resposta gerada é relevante ao tópico da pergunta feita e não apresenta alucinação de input. A resposta fornece informações precisas sobre como o Código Penal brasileiro trata o crime de dano qualificado por violências ou meios cruéis, respondendo à pergunta do usuário.
0.

A resposta gerada se mantev

In [13]:
has_hallucination_llama_ds

[{'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'O crime de difamação (art. 139, CP) consiste em atribuir a alguém fato ofensivo à sua reputação.\r\n\r\nProva da verdade: Se o acusado provar a veracidade do fato difamatório, a conduta será atípica (não constitui crime), exceto se:\r\n\r\nO fato se referir à vida privada da vítima;\r\n\r\nA divulgação for desnecessária ou motivada por malícia (art. 140, §1º, CP).',
  'input_hallucination_reason': 'A resposta gerada é relevante e direciona-se à pergunta feita sobre o crime de difamação, tratando do tema especificamente e fornecendo informações precisas sobre a legislação brasileira. Não há saída do tópico da pergunta, portanto não há alucinação de input.',
  'has_input_hallucination': '0.'},
 {'question': 'Qual a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência?',
  'answer': 'A injúria (art. 140, 

In [14]:
df_input_veredict_llama_ds = pd.DataFrame(has_hallucination_llama_ds)
df_input_veredict_llama_ds

,question,answer,input_hallucination_reason,has_input_hallucination
0,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",A resposta gerada é relevante e direciona-se à...,0.
1,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",Explicação: A resposta gerada é relevante e nã...,0\n
2,Quais práticas configuram o crime de falsifica...,"art. 297, CP\r\n\r\nConfigura-se quando alguém...",A resposta gerada está relacionada ao tema da ...,0.
3,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",A resposta gerada é relevante ao tópico da per...,0.
4,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,A resposta gerada se manteve dentro do tópico ...,0.
5,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Não houve alucinação de input.\n\nA resposta g...,0.
6,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\r\...,Não houve alucinação de input.\n\nA resposta f...,0.
7,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\r\n\r\nDefinição: Pri...",A resposta fornecida está diretamente relacion...,0.
8,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\r\n\...",Não há alucinação de input porque a resposta e...,0.
9,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,A resposta gerada não apresenta alucinação de ...,0.


In [15]:
df_input_veredict_llama_ds.to_csv("llm-output/input_hallucination_deepseek_llama.csv")

#### Gemini

In [ ]:
input_hallu = answerer(df_gemini_answers, model_llama, has_hallucination_llama_gm)
txt_processing_llama(has_hallucination_llama_gm)

0.
A resposta gerada está diretamente relacionada ao tópico da pergunta feita ("Como o código trata o crime de difamação quando há a eventual prova da verdade?"), não apresentando desvio ou mudança de assunto, o que sugere falta de alucinação de input.
0. Não houve alucinação de input.

A resposta fornece informações precisas e relevantes sobre a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência, o que é consistente com o tópico da pergunta. A resposta não apresenta informações alheias ao tema original, como nos exemplos de alucinação de input fornecidos.
0. Não houve alucinação de input.

A resposta gerada está diretamente relacionada ao tema da pergunta ("Quais práticas configuram o crime de falsificação de documento público?"), fornecendo informações precisas e relevantes sobre o crime de falsificação de documento público, especificando as condutas que configuram esse crime. A resposta não apresenta info

In [ ]:
has_hallucination_llama_gm

[{'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'No crime de difamação (art. 139 do Código Penal), a prova da verdade da imputação (chamada exceptio veritatis) geralmente não é admitida. Isso porque o bem jurídico protegido é a honra objetiva da pessoa, ou seja, a reputação perante terceiros. Mesmo que o fato difamatório seja verdadeiro, a sua divulgação pode ser considerada lesiva à reputação.\n\nNo entanto, o § 3º do artigo 139 traz uma exceção: a prova da verdade é admitida se a imputação se referir a fato de que o ofendido foi processado e absolvido judicialmente. Nesse caso específico, demonstrar a verdade da acusação original pode ser relevante.',
  'input_hallucination_reason': 'A resposta gerada não apresenta alucinação de input porque está diretamente relacionada ao tópico da pergunta feita. A discussão sobre o crime de difamação e a exceção à prova da verdade está dentro do contexto da pergunta original, não saiu do

In [ ]:
df_input_veredict_llama_gm = pd.DataFrame(has_hallucination_llama_gm)
df_input_veredict_llama_gm

,question,answer,input_hallucination_reason,has_input_hallucination
0,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,A resposta gerada não apresenta alucinação de ...,0\n
1,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...",Não houve alucinação de input porque a respost...,0.
2,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,A resposta gerada está relacionada ao tema da ...,0.
3,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,Não há alucinação de input porque a resposta e...,0.
4,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",Não houve alucinação de input.\n\nA resposta g...,0.
5,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,Não houve alucinação de input.\n\nA resposta g...,0.
6,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,A resposta gerada está diretamente relacionada...,0.
7,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,Não houve alucinação de input.\n\nA resposta g...,0.
8,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,A resposta fornecida está relacionada ao conte...,0.
9,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,A resposta foi relevante ao tópico da pergunta...,0.


In [ ]:
df_input_veredict_llama_gm.to_csv("llm-output/input_hallucination_gemini_llama.csv")

### GPT

In [ ]:
def txt_processing_gpt(answer_platform):
    for i in answer_platform:
        gpt_content = i["input_hallucination_reason"].content

        if gpt_content:
            has_input_hallucination = re.match(r"^([01])[.\s]", gpt_content)
            i["input_hallucination_reason"] = re.sub(r"content='.*.'|^[01]\.?\s*", "", gpt_content.strip())

            i["has_input_hallucination"] = has_input_hallucination[0]

In [ ]:
has_hallucination_gpt_ds = []
has_hallucination_gpt_gm = []

#### DeepSeek

In [ ]:
input_hallu = answerer(df_deepseek_answers, model_gpt, has_hallucination_gpt_ds)

txt_processing_gpt(has_hallucination_gpt_ds)

In [ ]:
has_hallucination_gpt_ds

[{'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'O crime de difamação (art. 139, CP) consiste em atribuir a alguém fato ofensivo à sua reputação.\n\nProva da verdade: Se o acusado provar a veracidade do fato difamatório, a conduta será atípica (não constitui crime), exceto se:\n\nO fato se referir à vida privada da vítima;\n\nA divulgação for desnecessária ou motivada por malícia (art. 140, §1º, CP).',
  'input_hallucination_reason': 'A resposta está diretamente relacionada ao tratamento do crime de difamação no código penal, explicando corretamente a questão da prova da verdade. Portanto, não houve alucinação de input.',
  'has_input_hallucination': '0 '},
 {'question': 'Qual a pena prevista para injúria quando há utilização de elementos referentes à religião ou condição de pessoa idosa ou com deficiência?',
  'answer': 'A injúria (art. 140, CP) é ofender a dignidade de alguém.\n\nQualificadora: \nSe a injúria utilizar eleme

In [ ]:
df_input_veredict_gpt_ds = pd.DataFrame(has_hallucination_gpt_ds)
df_input_veredict_gpt_ds

,question,answer,input_hallucination_reason,has_input_hallucination
0,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",A resposta está diretamente relacionada ao tra...,0
1,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",Houve resposta direta e pertinente à pergunta ...,0
2,Quais práticas configuram o crime de falsifica...,"art. 297, CP\n\nConfigura-se quando alguém:\n-...",Não houve alucinação de input porque a respost...,0.
3,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...","Não houve alucinação de input, pois a resposta...",0.
4,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,Não houve alucinação de input porque a respost...,0.
5,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Houve coerência entre a pergunta sobre a legis...,0
6,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\n\...,"Não houve alucinação de input, pois a resposta...",0.
7,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\n\nDefinição: Privar ...","Não houve alucinação de input, pois a resposta...",0.
8,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\n\nS...","Não houve alucinação de input, pois a resposta...",0.
9,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,A resposta está totalmente alinhada à pergunta...,0


In [ ]:
df_input_veredict_gpt_ds.to_csv("llm-output/input_hallucination_deepseek_gpt.csv")

#### Gemini

In [ ]:
input_hallu = answerer(df_gemini_answers, model_gpt, has_hallucination_gpt_gm)

txt_processing_gpt(has_hallucination_gpt_gm)

content='0 Houve pertinência entre a pergunta sobre como o código trata o crime de difamação diante da prova da verdade e a resposta fornecida, que explica o artigo 139 do Código Penal, a exceptio veritatis e a exceção do § 3º do artigo. Portanto, a resposta está dentro do tópico solicitado, sem fugir ao tema.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 410, 'total_tokens': 482, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_38647f5e19', 'id': 'chatcmpl-BZcAGnKrfOO5J1TqTWbT8XDeIBA9l', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--2e3bc047-8cf7-48a1-91b6-7786a066ad5b-0' usage_metadata={'input_tokens': 410, 'output_tokens': 72, 'total_tokens': 482, 'input_token_det

In [ ]:
has_hallucination_gpt_gm

[{'question': 'Como o código trata o crime de difamação quando há a eventual prova da verdade?',
  'answer': 'No crime de difamação (art. 139 do Código Penal), a prova da verdade da imputação (chamada exceptio veritatis) geralmente não é admitida. Isso porque o bem jurídico protegido é a honra objetiva da pessoa, ou seja, a reputação perante terceiros. Mesmo que o fato difamatório seja verdadeiro, a sua divulgação pode ser considerada lesiva à reputação.\n\nNo entanto, o § 3º do artigo 139 traz uma exceção: a prova da verdade é admitida se a imputação se referir a fato de que o ofendido foi processado e absolvido judicialmente. Nesse caso específico, demonstrar a verdade da acusação original pode ser relevante.',
  'input_hallucination_reason': 'Houve pertinência entre a pergunta sobre como o código trata o crime de difamação diante da prova da verdade e a resposta fornecida, que explica o artigo 139 do Código Penal, a exceptio veritatis e a exceção do § 3º do artigo. Portanto, a respo

In [ ]:
df_input_veredict_gpt_gm = pd.DataFrame(has_hallucination_gpt_gm)
df_input_veredict_gpt_gm

,question,answer,input_hallucination_reason,has_input_hallucination
0,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,Houve pertinência entre a pergunta sobre como ...,0
1,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...","Não houve alucinação de input, pois a resposta...",0.
2,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,"Não houve alucinação de input, pois a resposta...",0.
3,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,"Não houve alucinação de input, pois a resposta...",0.
4,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",Houve correspondência direta entre a pergunta ...,0
5,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,"Não houve alucinação de input, pois a resposta...",0.
6,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,Não houve alucinação de input porque a respost...,0.
7,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,Não houve alucinação de input pois a resposta ...,0.
8,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,Não houve alucinação de input porque a respost...,0.
9,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,A resposta está diretamente relacionada à perg...,0.


In [ ]:
df_input_veredict_gpt_gm.to_csv("llm-output/input_hallucination_gemini_gpt.csv")

## Alucinações contextuais

In [ ]:
#Alucinação contextual
def has_context_hallucination_llm(question, answer, model):
    contextHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de contexto.
        Considere que de alucinação de contexto é quando a resposta se contradiz.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0. Coloque o 0 ou o 1 entre parêntesis.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    prompt = ChatPromptTemplate.from_template(contextHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"question": question, "answer": answer })

    return result

## Alucinações factuais

In [ ]:
pdf_loader = PyPDFLoader("pdf/codigo_penal.pdf")
load_codigo_penal = pdf_loader.load()

In [ ]:
codigo_penal = ""

for page in load_codigo_penal:
    codigo_penal += page.page_content

In [ ]:
def has_factual_hallucination_llm(context, question, answer, vector, model):
    FactualHallucinationTemplate = """
        Você é um assistente criado para verificar se houveram alucinações factuais.
        Considere que uma alucinação é factual quando há uma informação falsa na resposta.

        Exemplo 1:
        Pergunta: Qual o maior animal do mundo?
        Resposta: O maior animal do mundo é a formiga

        Exemplo 2:
        Pergunta: Qual a cor do girassol?
        Resposta: Girassóis são roxos

        Exemplo 3: 
        Pergunta: Quantas rodas tem um carro?
        Resposta: Um carro possui 2 rodas.

        Caso haja alucinação factual, responda com 1. Caso não tenha alucinação de factual, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        Responda com base no contexto
        {context}
        
        Considerando que a resposta gerada para a pergunta "{question}" foi "{answer}", houve alucinação factual?
"""
    prompt_template = ChatPromptTemplate.from_template(FactualHallucinationTemplate)

    chain = prompt_template | model

    result = chain.invoke({"question": question, "answer": answer })

    return result

In [ ]:
txt_splitter = RecursiveCharacterTextSplitter(chunk_size=1400, 
                                              chunk_overlap=120, 
                                              length_function=len,
                                              separators=["\n\n", "\n", " "])

chunks = txt_splitter.split_text(text=codigo_penal)

In [ ]:
def has_factual_hallucination_llm():
    FactualHallucinationTemplate = """
        
"""
    prompt = ChatPromptTemplate.from_template(FactualHallucinationTemplate)
    return

### Llama 3

### GPT 